In [2]:
list_a = [1, 2, 3, 4, 5]
list_b = [6, 7, 8, 9, 10]


list_c = [x + y for x, y in zip(list_a, list_b)]
dict_c = {x: y for x, y in zip(list_a, list_b)}

In [3]:
list_c

[7, 9, 11, 13, 15]

In [4]:
def deco(func):
    def inner(func):
        print('running inner()')
    return inner  

@deco
def target():  
    print('running target()')

In [5]:
target = deco(target)

In [6]:
import pandas as pd
import numpy as np

# Sales data for multiple stores
sales = pd.DataFrame(
    {"revenue": [1000, 1200, 800, 950]},
    index=pd.Index(["LON", "NYC", "TOK", "SYD"], name="store_id"),
)

# Manager data, but only for a SUBSET of stores
managers = pd.DataFrame(
    {"manager": ["Alice", "Charlie"]}, index=pd.Index(["LON", "TOK"], name="store_id")
)

In [7]:
sales

,revenue
store_id,
LON,1000
NYC,1200
TOK,800
SYD,950


In [8]:
managers

,manager
store_id,
LON,Alice
TOK,Charlie


In [48]:
df = pd.DataFrame(
    {
        "unique_id": [1, 2, 3, 4],
        "col1": [1, 1, 2, 2],
        "col2": [2, 2, 3, 3],
        "col3": [3, 3, 4, 4],
        "col4": ["a", "a", "b", "b"],
        "col5": [True, True, False, False],
    }
)

# pids = df[(df["col1"] > 1) & (df["col2"] > 2)]['unique_id'].unique()

# df = df.assign(fid=np.where(df["unique_id"].isin(pids), True, False))

# df

In [10]:
match_condition = (df["col1"] > 1) & (df["col2"] > 2)

df = df.assign(
    sfid=match_condition.groupby(df["unique_id"]).transform("all")
)
df

,unique_id,col1,col2,col3,col4,col5,fid,sfid
0,1,1,2,3,a,True,False,False
1,2,1,2,3,a,True,False,False
2,3,2,3,4,b,False,True,True
3,4,2,3,4,b,False,True,True


In [11]:
match_condition

0    False
1    False
2     True
3     True
dtype: bool

In [12]:
"""

apply column and row wise

loc to change values in place





"""

'\n\napply column and row wise\n\nloc to change values in place\n\n\n\n\n\n'

In [75]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sqlalchemy
import os
import time

In [76]:
df = pd.DataFrame(
    {
        "col1": [1, 1, 2, 2],
        "col2": [2, 2, 3, 3],
        "col3": [3, 3, 4, 4],
        "col4": ["a", "a", "b", "b"],
        "col5": [True, True, False, False],
        "col6": [
            datetime(2000, 1, 1),
            datetime(2000, 1, 1),
            datetime(2000, 1, 1),
            datetime(2000, 1, 1),
        ],
    }
)


In [78]:
df.loc[df['col1'] > 1, 'col7'] = 999
df

,col1,col2,col3,col4,col5,col6,col7
0,1,2,3,a,True,2000-01-01,NaN
1,1,2,3,a,True,2000-01-01,NaN
2,2,999,4,b,False,2000-01-01,999.0
3,2,999,4,b,False,2000-01-01,999.0


In [52]:
df.groupby('col1')['col2'].transform('sum')

0    4
1    4
2    6
3    6
Name: col2, dtype: int64

In [53]:
df.groupby('col1').apply(lambda x: x.nlargest(1, 'col2'))

/var/folders/yg/tm584mqn4k12grwdd216v8540000gn/T/ipykernel_92070/231175614.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('col1').apply(lambda x: x.nlargest(1, 'col2'))


,,col1,col2,col3,col4,col5,col6
col1,,,,,,,
1,0,1,2,3,a,True,2000-01-01
2,2,2,3,4,b,False,2000-01-01


In [23]:
df.loc[:, "test_col"] = np.select(
    [
        df["col1"] >= 2,
        df["col1"] <= 1,
    ],
    [True, False],
)
df






/var/folders/yg/tm584mqn4k12grwdd216v8540000gn/T/ipykernel_92070/1951636299.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, "test_col"] = np.select(


,col1,col2,col3,col4,col5,col6,test_col
0,1,2,3,a,True,2000-01-01,0
1,1,2,3,a,True,2000-01-01,0
2,2,3,4,b,False,2000-01-01,1
3,2,3,4,b,False,2000-01-01,1


In [23]:
df

,col1,col2,col3,col4,col5,col6,test_col
0,1,2,3,a,True,2000-01-01,0
1,1,2,3,a,True,2000-01-01,0
2,2,3,4,b,False,2000-01-01,0
3,2,3,4,b,False,2000-01-01,0


In [20]:
df.loc[2, 'test_col'] = 1

In [37]:
COUNT = 0

In [40]:
def fake_apply(row):
    print(datetime.now())
    if row['col1'] >= 2:
        return 'hi'
    else:
        return 'bye'

In [43]:
df['test_apply'] = df.apply(fake_apply, axis=1)
    

2025-06-21 15:06:22.951086
2025-06-21 15:06:22.951163
2025-06-21 15:06:22.951185
2025-06-21 15:06:22.951204


In [31]:
df

,col1,col2,col3,col4,col5,col6,test_col,test_apply
0,1,2,3,a,True,2000-01-01,0,bye
1,1,2,3,a,True,2000-01-01,0,bye
2,2,3,4,b,False,2000-01-01,1,bye
3,2,3,4,b,False,2000-01-01,1,bye


In [44]:
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# COLUMN APPLY (axis=0) - function gets each column as Series
def col_func(series):
    print(f"Got series: {series.name}, values: {list(series)}")
    return series.sum()

result_col = df.apply(col_func, axis=0)
# Output:
# Got series: A, values: [1, 2, 3]
# Got series: B, values: [4, 5, 6] 
# Got series: C, values: [7, 8, 9]
# Result: A=6, B=15, C=24

# ROW APPLY (axis=1) - function gets each row as Series
def row_func(series):
    print(f"Got row {series.name}, values: {list(series)}")
    return series.sum()

result_row = df.apply(row_func, axis=1)
# Output:
# Got row 0, values: [1, 4, 7]
# Got row 1, values: [2, 5, 8]
# Got row 2, values: [3, 6, 9]
# Result: [12, 15, 18]

Got series: A, values: [1, 2, 3]
Got series: B, values: [4, 5, 6]
Got series: C, values: [7, 8, 9]
Got row 0, values: [1, 4, 7]
Got row 1, values: [2, 5, 8]
Got row 2, values: [3, 6, 9]


In [45]:
result_col

A     6
B    15
C    24
dtype: int64

In [46]:
result_row


0    12
1    15
2    18
dtype: int64

In [47]:
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [64]:
import time
import re

class TimestampedMeta(type):
    def __new__(mcs, name, bases, dct):
        print(mcs)
        print(name)
        print(bases)
        print(dct)
        # Check method names for snake_case
        for key, value in dct.items():
            if callable(value) and not key.startswith('_'):
                if not re.match(r'^[a-z][a-z0-9_]*$', key):
                    raise ValueError(f"Method '{key}' must be snake_case")
        
        # Add created_at timestamp
        dct['created_at'] = time.time()

        print('---------------------check-------------------------------')

        print(dct)
        
        return super().__new__(mcs, name, bases, dct)

# Usage
class MyModel(SomeParent, metaclass=TimestampedMeta):
    def get_data(self):  # Valid snake_case
        return "data"
    
    # def getData(self):  # Would raise ValueError
    #     return "data"

print(MyModel.created_at)  # timestamp when class was created

NameError: name 'SomeParent' is not defined

In [ ]:
class DatabaseManager:
    def __enter__(self):
        self.conn = get_connection()
        self.conn.begin_transaction()
        return self.conn  # This gets assigned to 'as' variable
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:  # No exception
            self.conn.commit()
        else:  # Exception occurred
            self.conn.rollback()
        self.conn.close()
        return False  # Don't suppress exceptions

# Usage
with DatabaseManager() as conn:
    conn.execute("INSERT ...")
    # If this raises exception, rollback happens
    # If no exception, commit happens
    # Connection always closes

In [60]:
my_context()

In [57]:
t = MyModel

In [58]:
t.created_at

1750515973.060721

In [74]:
# Regular yield - yields individual items
def simple_gen():
    yield 1
    yield 2
    yield 3

# yield from - delegates to another iterable
def delegating_gen():
    yield from [1, 2, 3]  # Equivalent to the above

# The power shows in nested structures:
def flatten_with_yield(items):
    for item in items:
        if isinstance(item, list):
            # This is WRONG - yields the generator object!
            yield flatten_with_yield(item)  # Oops!
        else:
            yield item

def flatten_with_yield_from(items):
    for item in items:
        if isinstance(item, list):
            yield from flatten_with_yield_from(item)  # Correct!
        else:
            yield item

# Test data
nested = [1, [2, [3, 4]], 5, [6]]

# Wrong way yields generator objects
list(flatten_with_yield(nested))  # [1, <generator>, 5, <generator>]

# Right way flattens properly  
# list(flatten_with_yield_from(nested))  # [1, 2, 3, 4, 5, 6]

[1,
 <generator object flatten_with_yield at 0x16c0bcf20>,
 5,
 <generator object flatten_with_yield at 0x16c0bf3e0>]

In [ ]:
def retry(max_attempts=3, delay=1.0):
    def decorator(func):
        def wrapper(*args, **kwargs):
            # Uses max_attempts, delay, func from outer scopes
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception:
                    time.sleep(delay)
        return wrapper
    return decorator

@retry(max_attempts=5, delay=2.0)
def test_func():
    return "hello"

# Let's inspect the closure:
print(test_func.__closure__)  
# (<cell at 0x...>, <cell at 0x...>, <cell at 0x...>)

print(test_func.__code__.co_freevars)  
# ('delay', 'func', 'max_attempts')

# Access the actual values:
for i, var_name in enumerate(test_func.__code__.co_freevars):
    print(f"{var_name}: {test_func.__closure__[i].cell_contents}")

# Output:
# delay: 2.0
# func: <function test_func at 0x...>
# max_attempts: 5

(<cell at 0x16c612110: float object at 0x16c59d410>, <cell at 0x16c613d30: function object at 0x16c380cc0>, <cell at 0x16c6137c0: int object at 0x102f1b758>)
('delay', 'func', 'max_attempts')
delay: 2.0
func: <function test_func at 0x16c380cc0>
max_attempts: 5


In [67]:
x = [1,2,3]

In [68]:
y = x

In [69]:
y

[1, 2, 3]

In [70]:
y[0] = 999

In [71]:
x

[999, 2, 3]

In [88]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @classmethod
    def from_birth_year(cls, name, birth_year):
        """Create Person from birth year instead of age"""
        from datetime import datetime
        age = datetime.now().year - birth_year
        return cls(name, age)
    
    @classmethod
    def from_string(cls, person_string):
        """Create Person from 'Name,Age' format"""
        name, age_str = person_string.split(',')
        return cls(name.strip(), int(age_str.strip()))
    
    def __repr__(self):
        return f"Person(name='{self.name}', age{type(self.age)}={self.age})"

# Usage
p1 = Person.from_birth_year("Alice", 1990)
p2 = Person.from_string("Bob, 25")
print(str(p1))  # Person(name='Alice', age=34)
print(p2)  # Person(name='Bob', age=25)

Person(name='Alice', age<class 'int'>=35)
Person(name='Bob', age<class 'int'>=25)


In [84]:
p2

Person(name='Bob', age=25)

In [2]:

import pandas as pd

# Initialize the paramaters of the loan
loan_amount = 18000
apr = 5.29
loan_term = 60

# Get a monthly percentage rate
apr /= 100
mpr = apr / 12

# Calculate the Monthly Payment for a loan
monthly_payment = (loan_amount * mpr)/(1-(1+mpr) ** -loan_term)

def make_payment(principal, mpr, monthly_payment):
    '''Makes a 'payment' by subtracting and updated payment amount from the 
    principal. Returns the principal remaining, and the amount of principal and interest paid
    '''

    current_interest_payment = principal * mpr
    current_principal_payment = monthly_payment - current_interest_payment
    
    principal -= current_principal_payment
    
    return [round(principal, 2), round(current_principal_payment, 2), round(current_interest_payment, 2)] 

def main(principal, term_remaining, monthly_payment, mpr):
    '''Returns an Amortization Table in the form of a DataFrame
    '''
    
    payments = [[principal, 0, 0, 0]]
    total_interest = 0
    
    while principal > 0 and term_remaining > 0:
        payment = make_payment(principal, mpr, monthly_payment)
        principal = payment[0]
        term_remaining -= 1
        total_interest += payment[2]
        payment.append(total_interest)
        payments.append(payment)

    amortization_table = pd.DataFrame(data=payments,
                                      columns=['Principal Remaining',
                                              'Current Principal Payment',
                                              'Current Interest Payment',
                                              'Total Interest Paid'])
    return amortization_table


main(loan_amount, loan_term, monthly_payment, mpr)


,Principal Remaining,Current Principal Payment,Current Interest Payment,Total Interest Paid
0,18000.00,0.00,0.00,0.00
1,17737.27,262.73,79.35,79.35
2,17473.38,263.89,78.19,157.54
3,17208.33,265.05,77.03,234.57
4,16942.11,266.22,75.86,310.43
...,...,...,...,...
56,1353.36,334.64,7.44,2509.79
57,1017.25,336.11,5.97,2515.76
58,679.66,337.59,4.48,2520.24
59,340.58,339.08,3.00,2523.24
